## 🎉 Workflow Complete!

### What You've Accomplished:

1. **🏍️ Motorcycle Selection**: Chose from predefined motorcycles or built a custom configuration
2. **🗺️ Track Testing**: Selected tracks for comprehensive testing
3. **🚀 Simulation**: Ran physics-based powertrain simulations  
4. **📊 Analysis**: Viewed detailed performance results
5. **💾 Export**: Generated comprehensive JSON files with all specifications and results

### Key Features:

- **🖱️ No-Code Interface**: Complete workflow using only dropdowns and buttons
- **📋 Comprehensive Data**: Full motorcycle specifications plus simulation results
- **🔧 Engineering Focus**: All data needed for powertrain analysis and optimization
- **☁️ Cloud Compatible**: Runs in Google Colab and local Jupyter environments
- **📈 Professional Output**: Engineering-grade results and visualizations

### Next Steps:

- **🔄 Iterate Designs**: Test different motorcycle configurations
- **📊 Analyze Results**: Use exported JSON for detailed analysis
- **🏭 Scale Testing**: Run batch simulations across multiple tracks
- **📝 Document Findings**: Results ready for technical reports

The exported JSON contains everything needed for engineering analysis, including complete motorcycle specifications, component details, and performance metrics across all tested tracks.

# 🏍️ Ithaka Motorcycle Powertrain Designer

Welcome to the **streamlined motorcycle powertrain analysis tool**! This notebook is designed for non-technical users to easily:

- 🏍️ **Select motorcycles** from our predefined library or build custom configurations
- 🗺️ **Choose test tracks** from real GPS data
- 🚀 **Run simulations** with a single click
- 📊 **View results** with professional visualizations
- 💾 **Export data** for further analysis

**No coding required!** Just follow the steps and use the dropdown menus and buttons.

In [ ]:
# Setup and initialization (hidden from user)
import sys
import os
import warnings
warnings.filterwarnings('ignore')

# Check if we're in Colab and need to install
try:
    import google.colab
    IN_COLAB = True
    print("📦 Setting up environment for Google Colab...")
    
    # Clone repository if not already present
    if not os.path.exists('ithaka_powertrain_sim'):
        os.system('git clone https://github.com/Teodus/ithaka-powertrain-sim.git')
        os.chdir('ithaka-powertrain-sim')
        os.system('pip install -e .')
    else:
        print("✅ Repository already cloned")
        
except ImportError:
    IN_COLAB = False
    print("💻 Running in local environment")

# Import all required packages
try:
    import pandas as pd
    import numpy as np
    import matplotlib.pyplot as plt
    import ipywidgets as widgets
    from IPython.display import display, clear_output, HTML
    import json
    import glob
    from datetime import datetime
    
    # Import project modules
    from ithaka_powertrain_sim.component_library import (
        get_motorcycle_list, get_motorcycle_specs, create_motorcycle,
        ComponentPickerState, PREDEFINED_MOTORCYCLES
    )
    from ithaka_powertrain_sim.motorbike import Motorbike
    from ithaka_powertrain_sim.trajectory import load_gpx, append_and_resample_dataframe
    
    print("✅ All packages loaded successfully!")
    print(f"📊 Found {len(get_motorcycle_list())} predefined motorcycles")
    
    # Find GPX files
    gpx_files = glob.glob('docs/gpx_files/*.gpx')
    print(f"🗺️ Found {len(gpx_files)} test tracks")
    
except Exception as e:
    print(f"❌ Setup error: {e}")
    print("Please ensure all required packages are installed.")

In [ ]:
# Application state and UI classes (hidden implementation)
class MotorcyclePowertrainApp:
    def __init__(self):
        self.selected_motorcycle = None
        self.custom_motorcycle = None
        self.workflow_choice = None
        self.selected_tracks = []
        self.simulation_results = {}
        self.picker_state = None
        
    def get_available_tracks(self):
        """Get list of available GPX track files with user-friendly names"""
        gpx_files = glob.glob('docs/gpx_files/*.gpx')
        if not gpx_files:
            return [("No tracks found", None)]
        
        track_names = []
        for gpx_file in gpx_files:
            # Create user-friendly name from filename
            name = os.path.basename(gpx_file).replace('.gpx', '').replace('-', ' ').replace('_', ' ')
            name = ' '.join(word.capitalize() for word in name.split())
            track_names.append((name, gpx_file))
        
        return sorted(track_names)
    
    def simulate_motorcycle_on_track(self, motorcycle, track_file, track_name):
        """Run simulation for motorcycle on single track"""
        try:
            # Load and process trajectory
            trajectory_df = load_gpx(track_file)
            trajectory_df = append_and_resample_dataframe(trajectory_df)
            
            # Extract simulation parameters
            target_speed = trajectory_df["Target Speed"].to_list()
            delta_distance = np.diff(trajectory_df["Distance"], prepend=0).tolist()
            delta_elevation = np.diff(trajectory_df["Elevation"], prepend=0).tolist()
            approximate_time = trajectory_df["Target Time"].to_list()
            
            # Run step-by-step simulation
            achieved_speeds = [trajectory_df["Target Speed"].iloc[0]]
            reporting_dataframe_rows = []
            
            for index in range(1, len(trajectory_df)):
                delta_time = approximate_time[index] - approximate_time[index - 1]
                
                achieved_speed, reporting_dataframe_row = motorcycle.calculate_achieved_speed(
                    achieved_speeds[index - 1], target_speed[index], delta_time,
                    delta_distance[index], delta_elevation[index]
                )
                
                achieved_speeds.append(achieved_speed)
                reporting_dataframe_rows.append(reporting_dataframe_row)
                
                # Adjust timing for speed differences
                if index < len(trajectory_df) - 1 and achieved_speed != 0 and target_speed[index] != 0:
                    future_delta_time = (delta_distance[index + 1] / achieved_speed) - (
                        delta_distance[index + 1] / target_speed[index]
                    )
                    for j in range(index + 1, len(approximate_time)):
                        approximate_time[j] += future_delta_time
            
            # Combine results
            motorbike_dataframe = trajectory_df.copy()
            motorbike_dataframe["Approximate Time"] = approximate_time
            motorbike_dataframe["Achieved Speed"] = achieved_speeds
            
            if reporting_dataframe_rows:
                reporting_dataframe = pd.concat(reporting_dataframe_rows, ignore_index=True)
                final_results = motorbike_dataframe.join(reporting_dataframe)
            else:
                final_results = motorbike_dataframe
            
            # Calculate metrics
            total_time = approximate_time[-1] if approximate_time else 0
            total_distance = trajectory_df['Distance'].iloc[-1] if len(trajectory_df) > 0 else 0
            avg_speed = (total_distance / total_time * 3.6) if total_time > 0 else 0
            
            energy_consumed = 0
            if reporting_dataframe_rows and 'Energy Consumed (J)' in reporting_dataframe.columns:
                energy_consumed = reporting_dataframe['Energy Consumed (J)'].sum()
            
            return {
                'track_name': track_name,
                'track_file': track_file,
                'total_time_s': total_time,
                'total_distance_km': total_distance / 1000,
                'average_speed_kmh': avg_speed,
                'energy_consumed_kWh': energy_consumed / 3.6e6,
                'simulation_results': final_results,
                'trajectory': trajectory_df,
                'success': True
            }
            
        except Exception as e:
            return {
                'track_name': track_name,
                'track_file': track_file,
                'error': str(e),
                'success': False
            }

# Initialize the application
app = MotorcyclePowertrainApp()
print("🚀 Application initialized successfully!")

## Step 1: Choose Your Design Approach

Select how you want to create your motorcycle:

In [ ]:
# Step 1: Workflow Selection Interface
style = {'description_width': '150px'}
layout_wide = widgets.Layout(width='500px')

workflow_selector = widgets.RadioButtons(
    options=[
        ('🏍️ Choose from Predefined Motorcycles', 'predefined'),
        ('🔧 Build Custom Motorcycle', 'custom')
    ],
    value=None,
    description='Design Method:',
    style=style,
    layout=layout_wide
)

workflow_output = widgets.Output()

def handle_workflow_selection(change):
    app.workflow_choice = change.new
    with workflow_output:
        clear_output()
        if change.new == 'predefined':
            print("✅ You selected: Predefined Motorcycles")
            print("👇 Continue to Step 2 to choose a motorcycle")
        elif change.new == 'custom':
            print("✅ You selected: Custom Motorcycle Builder")
            print("👇 Continue to Step 2 to configure components")

workflow_selector.observe(handle_workflow_selection, names='value')

display(workflow_selector)
display(workflow_output)

## Step 2: Motorcycle Configuration

Configure your motorcycle based on your chosen approach:

In [ ]:
# Step 2: Motorcycle Selection and Configuration Interface

# Create all UI elements
motorcycle_selector = widgets.Dropdown(
    options=[("Select a motorcycle...", None)],
    value=None,
    description='Motorcycle:',
    style=style,
    layout=layout_wide
)

motorcycle_info = widgets.Output()
confirm_selection_btn = widgets.Button(
    description='✅ Select This Motorcycle',
    button_style='success',
    layout=widgets.Layout(width='200px')
)

# Custom motorcycle interface
custom_info = widgets.Output()
custom_interface = widgets.Output()
build_custom_btn = widgets.Button(
    description='🔧 Build Custom Motorcycle',
    button_style='primary',
    layout=widgets.Layout(width='200px')
)

# Update motorcycle dropdown when workflow is selected
def update_motorcycle_options():
    if app.workflow_choice == 'predefined':
        try:
            options = [("Select a motorcycle...", None)]
            for name in get_motorcycle_list():
                specs = get_motorcycle_specs(name)
                motorcycle_type = specs.get('type', 'Unknown')
                options.append((f"{name} ({motorcycle_type})", name))
            motorcycle_selector.options = options
        except Exception as e:
            print(f"Error loading motorcycles: {e}")

def display_motorcycle_info(change):
    if change.new is None or app.workflow_choice != 'predefined':
        return
    
    with motorcycle_info:
        clear_output()
        try:
            specs = get_motorcycle_specs(change.new)
            bike = create_motorcycle(change.new)
            
            print(f"📋 {change.new} Specifications")
            print("=" * 40)
            print(f"Type: {specs.get('type', 'Unknown')}")
            print(f"Total Mass: {bike.mass:.1f} kg")
            
            # Calculate total power
            total_power = 0
            for component in bike.child_components:
                if hasattr(component, '_maximum_power_generation'):
                    total_power += component._maximum_power_generation
            
            if total_power > 0:
                print(f"Max Power: {total_power / 1000:.1f} kW")
                print(f"Power-to-Weight: {(total_power / 1000) / bike.mass:.3f} kW/kg")
            
            if 'battery_kWh' in specs:
                print(f"Battery: {specs['battery_kWh']} kWh")
                if 'range_km' in specs:
                    print(f"Range: {specs['range_km']} km")
            
            if 'displacement_cc' in specs:
                print(f"Engine: {specs['displacement_cc']} cc")
                if 'fuel_L' in specs:
                    print(f"Fuel: {specs['fuel_L']} L")
            
            print(f"\\nDescription: {specs.get('description', 'No description available')}")
            
        except Exception as e:
            print(f"Error loading motorcycle info: {e}")

def confirm_motorcycle_selection(btn):
    if motorcycle_selector.value and app.workflow_choice == 'predefined':
        try:
            app.selected_motorcycle = create_motorcycle(motorcycle_selector.value)
            with motorcycle_info:
                clear_output()
                print(f"✅ {motorcycle_selector.value} selected successfully!")
                print("👇 Continue to Step 3 to choose test tracks")
        except Exception as e:
            with motorcycle_info:
                clear_output()
                print(f"❌ Error selecting motorcycle: {e}")

def setup_custom_interface(btn):
    if app.workflow_choice == 'custom':
        with custom_interface:
            clear_output()
            try:
                app.picker_state = ComponentPickerState()
                app.picker_state.display_interface()
                
                # Add build button after interface
                def build_custom_motorcycle(build_btn):
                    with custom_info:
                        clear_output()
                        try:
                            # Build from picker state
                            components = []
                            
                            # Add selected components
                            if app.picker_state.selected_engine:
                                components.append(app.picker_state.selected_engine)
                            if app.picker_state.selected_motor:
                                components.append(app.picker_state.selected_motor)
                            if app.picker_state.selected_battery:
                                components.append(app.picker_state.selected_battery)
                            if app.picker_state.selected_fuel_tank:
                                components.append(app.picker_state.selected_fuel_tank)
                                
                            if not components:
                                print("❌ No components selected. Please select at least one component.")
                                return
                            
                            # Create custom motorcycle
                            app.custom_motorcycle = Motorbike(
                                name=f"Custom_{datetime.now().strftime('%H%M%S')}",
                                child_components=components,
                                dry_mass_excluding_components=150.0,  # Base motorcycle mass
                                front_mass_ratio=0.4,
                                wheelbase=1.4,
                                front_wheel_radius=0.3,
                                rear_wheel_radius=0.3,
                                frontal_area=0.6,
                                coefficient_of_aerodynamic_drag=0.6
                            )
                            
                            print(f"✅ Custom motorcycle built successfully!")
                            print(f"Total Mass: {app.custom_motorcycle.mass:.1f} kg")
                            print(f"Components: {len(components)}")
                            print("👇 Continue to Step 3 to choose test tracks")
                            
                        except Exception as e:
                            print(f"❌ Error building motorcycle: {e}")
                
                final_build_btn = widgets.Button(
                    description='🏗️ Build My Motorcycle',
                    button_style='success',
                    layout=widgets.Layout(width='200px')
                )
                final_build_btn.on_click(build_custom_motorcycle)
                display(final_build_btn)
                
            except Exception as e:
                print(f"Error setting up custom interface: {e}")

# Connect event handlers
motorcycle_selector.observe(display_motorcycle_info, names='value')
confirm_selection_btn.on_click(confirm_motorcycle_selection)
build_custom_btn.on_click(setup_custom_interface)

# Display appropriate interface based on workflow choice
def show_motorcycle_interface(workflow_choice):
    if workflow_choice == 'predefined':
        update_motorcycle_options()
        display(motorcycle_selector)
        display(motorcycle_info)
        display(confirm_selection_btn)
        print("\\n" + "="*50)
        
    elif workflow_choice == 'custom':
        display(build_custom_btn)
        display(custom_interface)
        display(custom_info)
        print("\\n" + "="*50)
    else:
        print("⚠️ Please select a design method in Step 1 first")

# Show interface if workflow is already selected
if hasattr(app, 'workflow_choice') and app.workflow_choice:
    show_motorcycle_interface(app.workflow_choice)
else:
    print("⚠️ Please complete Step 1 first")

## Step 3: Select Test Tracks

Choose which tracks to test your motorcycle on:

In [ ]:
# Step 3: Track Selection Interface

track_mode_selector = widgets.RadioButtons(
    options=[
        ('🎯 Test Single Track', 'single'),
        ('🌍 Test All Available Tracks', 'all')
    ],
    value=None,
    description='Test Mode:',
    style=style,
    layout=layout_wide
)

single_track_selector = widgets.Dropdown(
    options=[("Select a track...", None)],
    value=None,
    description='Track:',
    style=style,
    layout=layout_wide
)

track_info = widgets.Output()
confirm_tracks_btn = widgets.Button(
    description='✅ Confirm Track Selection',
    button_style='success',
    layout=widgets.Layout(width='200px')
)

def update_track_options():
    """Update track dropdown with available tracks"""
    try:
        available_tracks = app.get_available_tracks()
        if available_tracks and available_tracks[0][1] is not None:
            options = [("Select a track...", None)] + available_tracks
            single_track_selector.options = options
        else:
            single_track_selector.options = [("No tracks available", None)]
    except Exception as e:
        print(f"Error loading tracks: {e}")

def handle_track_mode_change(change):
    with track_info:
        clear_output()
        if change.new == 'single':
            update_track_options()
            single_track_selector.layout.display = 'block'
            print("📍 Select a single track to test on")
        elif change.new == 'all':
            single_track_selector.layout.display = 'none'
            available_tracks = app.get_available_tracks()
            if available_tracks and available_tracks[0][1] is not None:
                print(f"🌍 Will test on all {len(available_tracks)} available tracks:")
                for name, _ in available_tracks[:5]:  # Show first 5
                    print(f"  • {name}")
                if len(available_tracks) > 5:
                    print(f"  ... and {len(available_tracks) - 5} more")
            else:
                print("❌ No tracks found")

def handle_single_track_change(change):
    if change.new and track_mode_selector.value == 'single':
        with track_info:
            clear_output()
            print(f"📍 Selected: {change.new[0]}")

def confirm_track_selection(btn):
    current_motorcycle = app.selected_motorcycle or app.custom_motorcycle
    if not current_motorcycle:
        with track_info:
            clear_output()
            print("❌ Please select a motorcycle first (Step 2)")
        return
    
    if not track_mode_selector.value:
        with track_info:
            clear_output()
            print("❌ Please select a test mode")
        return
    
    if track_mode_selector.value == 'single' and not single_track_selector.value:
        with track_info:
            clear_output()
            print("❌ Please select a track")
        return
    
    # Store selection
    if track_mode_selector.value == 'single':
        app.selected_tracks = [single_track_selector.value]
    else:
        app.selected_tracks = app.get_available_tracks()
    
    with track_info:
        clear_output()
        if track_mode_selector.value == 'single':
            print(f"✅ Track selected: {app.selected_tracks[0][0]}")
        else:
            print(f"✅ All {len(app.selected_tracks)} tracks selected")
        print("👇 Continue to Step 4 to run simulation")

# Connect event handlers
track_mode_selector.observe(handle_track_mode_change, names='value')
single_track_selector.observe(handle_single_track_change, names='value')
confirm_tracks_btn.on_click(confirm_track_selection)

# Hide single track selector initially
single_track_selector.layout.display = 'none'

# Check if we have a motorcycle selected
current_motorcycle = app.selected_motorcycle or app.custom_motorcycle
if current_motorcycle:
    display(track_mode_selector)
    display(single_track_selector)
    display(track_info)
    display(confirm_tracks_btn)
else:
    print("⚠️ Please complete Step 2 (select/build a motorcycle) first")

In [ ]:
## Step 4: Run Simulation

Execute the physics-based simulation with your motorcycle and tracks:

# Step 4: Simulation Execution Interface

run_simulation_btn = widgets.Button(
    description='🚀 Run Simulation',
    button_style='primary',
    layout=widgets.Layout(width='200px', height='40px')
)

simulation_progress = widgets.Output()

def run_simulation(btn):
    # Check prerequisites
    current_motorcycle = app.selected_motorcycle or app.custom_motorcycle
    if not current_motorcycle:
        with simulation_progress:
            clear_output()
            print("❌ No motorcycle selected. Complete Step 2 first.")
        return
    
    if not app.selected_tracks:
        with simulation_progress:
            clear_output()
            print("❌ No tracks selected. Complete Step 3 first.")
        return
    
    with simulation_progress:
        clear_output()
        print("🚀 Starting simulation...")
        print(f"🏍️ Motorcycle: {current_motorcycle.name}")
        print(f"📊 Tracks to test: {len(app.selected_tracks)}")
        print("=" * 50)
        
        app.simulation_results = {}
        successful_runs = 0
        
        for i, (track_name, track_file) in enumerate(app.selected_tracks):
            if track_file is None:
                continue
                
            print(f"📍 ({i+1}/{len(app.selected_tracks)}) Testing: {track_name}")
            
            try:
                result = app.simulate_motorcycle_on_track(current_motorcycle, track_file, track_name)
                app.simulation_results[track_name] = result
                
                if result['success']:
                    successful_runs += 1
                    print(f"  ✅ Completed in {result['total_time_s']/60:.1f} min")
                    print(f"     Distance: {result['total_distance_km']:.1f} km")
                    print(f"     Avg Speed: {result['average_speed_kmh']:.1f} km/h")
                    if result['energy_consumed_kWh'] > 0:
                        print(f"     Energy Used: {result['energy_consumed_kWh']:.2f} kWh")
                else:
                    print(f"  ❌ Failed: {result.get('error', 'Unknown error')}")
            
            except Exception as e:
                print(f"  ❌ Exception: {str(e)}")
                app.simulation_results[track_name] = {
                    'track_name': track_name,
                    'error': str(e),
                    'success': False
                }
        
        print("=" * 50)
        print(f"🎉 Simulation Complete!")
        print(f"✅ Successful: {successful_runs}/{len(app.selected_tracks)} tracks")
        
        if successful_runs > 0:
            print("👇 Continue to Step 5 to view detailed results and export data")
        else:
            print("❌ No successful simulations. Check track files and motorcycle configuration.")

run_simulation_btn.on_click(run_simulation)

# Only show if prerequisites are met
current_motorcycle = app.selected_motorcycle or app.custom_motorcycle
if current_motorcycle and app.selected_tracks:
    display(run_simulation_btn)
    display(simulation_progress)
else:
    missing = []
    if not current_motorcycle:
        missing.append("motorcycle (Step 2)")
    if not app.selected_tracks:
        missing.append("tracks (Step 3)")
    
    print(f"⚠️ Complete the following first: {', '.join(missing)}")

In [ ]:
## Step 5: Results & Export

View simulation results and export comprehensive data:

# Step 5: Results Analysis and Export Interface

view_results_btn = widgets.Button(
    description='📊 View Detailed Results',
    button_style='info',
    layout=widgets.Layout(width='200px')
)

export_results_btn = widgets.Button(
    description='💾 Export Results (JSON)',
    button_style='success',
    layout=widgets.Layout(width='200px')
)

results_display = widgets.Output()
export_status = widgets.Output()

def view_detailed_results(btn):
    if not app.simulation_results:
        with results_display:
            clear_output()
            print("❌ No simulation results available. Run simulation first.")
        return
    
    with results_display:
        clear_output()
        
        # Get motorcycle info
        current_motorcycle = app.selected_motorcycle or app.custom_motorcycle
        
        print("📊 SIMULATION RESULTS SUMMARY")
        print("=" * 60)
        
        if current_motorcycle:
            print(f"🏍️ Motorcycle: {current_motorcycle.name}")
            print(f"⚖️ Total Mass: {current_motorcycle.mass:.1f} kg")
            
            # Calculate total power
            total_power = 0
            for component in current_motorcycle.child_components:
                if hasattr(component, '_maximum_power_generation'):
                    total_power += component._maximum_power_generation
            
            if total_power > 0:
                power_to_weight = (total_power / 1000) / current_motorcycle.mass
                print(f"⚡ Total Power: {total_power / 1000:.1f} kW")
                print(f"🏁 Power-to-Weight Ratio: {power_to_weight:.3f} kW/kg")
            print()
        
        # Results table
        print("📈 TRACK PERFORMANCE RESULTS:")
        print("-" * 85)
        print(f"{'Track Name':<30} {'Distance':<10} {'Time':<8} {'Avg Speed':<12} {'Energy':<10}")
        print(f"{'':30} {'(km)':<10} {'(min)':<8} {'(km/h)':<12} {'(kWh)':<10}")
        print("-" * 85)
        
        successful_results = []
        for track_name, result in app.simulation_results.items():
            if result.get('success'):
                dist = result['total_distance_km']
                time_min = result['total_time_s'] / 60
                speed = result['average_speed_kmh']
                energy = result['energy_consumed_kWh']
                print(f"{track_name:<30} {dist:<10.1f} {time_min:<8.1f} {speed:<12.1f} {energy:<10.2f}")
                successful_results.append(result)
            else:
                error_msg = result.get('error', 'Unknown error')[:20] + "..."
                print(f"{track_name:<30} {'ERROR':<10} {'---':<8} {'---':<12} {'---':<10}")
        
        print("-" * 85)
        
        # Summary statistics
        if successful_results:
            avg_speed = np.mean([r['average_speed_kmh'] for r in successful_results])
            total_distance = sum([r['total_distance_km'] for r in successful_results])
            total_energy = sum([r['energy_consumed_kWh'] for r in successful_results])
            total_time = sum([r['total_time_s'] for r in successful_results]) / 3600  # hours
            
            print(f"\\n📊 SUMMARY STATISTICS:")
            print(f"✅ Successful simulations: {len(successful_results)}/{len(app.simulation_results)}")
            print(f"🏁 Average speed across all tracks: {avg_speed:.1f} km/h")
            print(f"🛣️ Total distance simulated: {total_distance:.1f} km")
            print(f"⏱️ Total simulation time: {total_time:.1f} hours")
            print(f"🔋 Total energy consumed: {total_energy:.2f} kWh")
            
            if total_distance > 0:
                efficiency = total_energy / total_distance
                print(f"⚡ Average energy efficiency: {efficiency:.3f} kWh/km")
                
                if total_energy > 0:
                    range_per_kwh = total_distance / total_energy
                    print(f"📏 Range per kWh: {range_per_kwh:.1f} km/kWh")

def export_simulation_results(btn):
    if not app.simulation_results:
        with export_status:
            clear_output()
            print("❌ No results to export. Run simulation first.")
        return
    
    current_motorcycle = app.selected_motorcycle or app.custom_motorcycle
    if not current_motorcycle:
        with export_status:
            clear_output()
            print("❌ No motorcycle data available.")
        return
    
    try:
        with export_status:
            clear_output()
            print("💾 Preparing export...")
        
        # Create comprehensive export data
        export_data = {
            'export_metadata': {
                'timestamp': datetime.now().isoformat(),
                'workflow_type': app.workflow_choice,
                'application_version': '1.0',
                'total_tracks_tested': len(app.simulation_results)
            },
            'motorcycle_specifications': {
                'name': current_motorcycle.name,
                'total_mass_kg': current_motorcycle.mass,
                'dry_mass_excluding_components_kg': current_motorcycle.dry_mass_excluding_components,
                'front_mass_ratio': current_motorcycle.front_mass_ratio,
                'front_wheel_radius_m': current_motorcycle.front_wheel_radius,
                'rear_wheel_radius_m': current_motorcycle.rear_wheel_radius,
                'frontal_area_m2': current_motorcycle.frontal_area,
                'coefficient_of_aerodynamic_drag': current_motorcycle.coefficient_of_aerodynamic_drag,
                'components': []
            },
            'simulation_results': {},
            'performance_summary': {}
        }
        
        # Add component specifications
        total_power = 0
        for component in current_motorcycle.child_components:
            comp_spec = {
                'name': component.name,
                'type': type(component).__name__,
                'mass_kg': component.mass
            }
            
            if hasattr(component, '_maximum_power_generation'):
                power_w = component._maximum_power_generation
                comp_spec['max_power_W'] = power_w
                comp_spec['max_power_kW'] = power_w / 1000
                total_power += power_w
            
            if hasattr(component, 'remaining_energy_capacity'):
                comp_spec['energy_capacity_kWh'] = component.remaining_energy_capacity / 3.6e6
                
            export_data['motorcycle_specifications']['components'].append(comp_spec)
        
        export_data['motorcycle_specifications']['total_power_kW'] = total_power / 1000
        if current_motorcycle.mass > 0:
            export_data['motorcycle_specifications']['power_to_weight_ratio'] = (total_power / 1000) / current_motorcycle.mass
        
        # Add simulation results
        successful_results = []
        for track_name, result in app.simulation_results.items():
            if result.get('success'):
                export_data['simulation_results'][track_name] = {
                    'track_name': result['track_name'],
                    'track_file': result['track_file'],
                    'total_time_s': result['total_time_s'],
                    'total_distance_km': result['total_distance_km'],
                    'average_speed_kmh': result['average_speed_kmh'],
                    'energy_consumed_kWh': result['energy_consumed_kWh'],
                    'status': 'success'
                }
                successful_results.append(result)
            else:
                export_data['simulation_results'][track_name] = {
                    'track_name': result['track_name'],
                    'track_file': result.get('track_file', 'unknown'),
                    'error': result.get('error', 'Unknown error'),
                    'status': 'failed'
                }
        
        # Add performance summary
        if successful_results:
            export_data['performance_summary'] = {
                'successful_simulations': len(successful_results),
                'failed_simulations': len(app.simulation_results) - len(successful_results),
                'average_speed_kmh': float(np.mean([r['average_speed_kmh'] for r in successful_results])),
                'total_distance_km': float(sum([r['total_distance_km'] for r in successful_results])),
                'total_time_hours': float(sum([r['total_time_s'] for r in successful_results]) / 3600),
                'total_energy_consumed_kWh': float(sum([r['energy_consumed_kWh'] for r in successful_results])),
            }
            
            total_dist = export_data['performance_summary']['total_distance_km']
            total_energy = export_data['performance_summary']['total_energy_consumed_kWh']
            
            if total_dist > 0 and total_energy > 0:
                export_data['performance_summary']['energy_efficiency_kWh_per_km'] = total_energy / total_dist
                export_data['performance_summary']['range_per_kWh_km'] = total_dist / total_energy
        
        # Generate filename
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        motorcycle_name = current_motorcycle.name.replace(' ', '_').replace('/', '_').lower()
        filename = f"ithaka_simulation_{motorcycle_name}_{timestamp}.json"
        
        # Write to file
        with open(filename, 'w') as f:
            json.dump(export_data, f, indent=2, default=str)
        
        with export_status:
            clear_output()
            print("✅ Export completed successfully!")
            print(f"📄 Filename: {filename}")
            print("\\n📊 Export contains:")
            print("  • Complete motorcycle specifications")
            print("  • Component details and power ratings")
            print(f"  • {len(app.simulation_results)} track simulation results")
            print("  • Performance summary and efficiency metrics")
            print("  • Metadata and timestamps")
            
            file_size = os.path.getsize(filename) / 1024  # KB
            print(f"  • File size: {file_size:.1f} KB")
            
    except Exception as e:
        with export_status:
            clear_output()
            print(f"❌ Export failed: {str(e)}")

# Connect button handlers
view_results_btn.on_click(view_detailed_results)
export_results_btn.on_click(export_simulation_results)

# Only show if simulation has been run
if app.simulation_results:
    display(widgets.HBox([view_results_btn, export_results_btn]))
    display(results_display)
    display(export_status)
else:
    print("⚠️ Run simulation first (Step 4) to view and export results")

# 📊 Step 5: Results Analysis & Export

View simulation results and export comprehensive data:

In [ ]:
# Export functionality
export_json_btn = widgets.Button(
    description='💾 Export Results as JSON',
    button_style='success',
    layout=widgets.Layout(width='250px')
)

export_output = widgets.Output()

def export_results_to_json(btn):
    """Export comprehensive results including motorcycle build and simulation data"""
    if not simulation_results:
        with export_output:
            clear_output()
            print("❌ No simulation results to export.")
        return
    
    current_motorcycle = selected_motorcycle or custom_motorcycle
    if not current_motorcycle:
        with export_output:
            clear_output()
            print("❌ No motorcycle data to export.")
        return
    
    try:
        # Create comprehensive export data
        export_data = {
            'export_info': {
                'timestamp': datetime.now().isoformat(),
                'workflow_type': workflow_choice,
                'simulation_mode': track_mode_selector.value,
            },
            'motorcycle_specification': create_motorcycle_specification_dict(current_motorcycle),
            'simulation_results': {},
            'summary_statistics': {}
        }
        
        # Add simulation results (excluding non-serializable objects)
        for track_name, result in simulation_results.items():
            if 'error' in result:
                export_data['simulation_results'][track_name] = {
                    'track_name': result['track_name'],
                    'track_file': result['track_file'],
                    'error': result['error']
                }
            else:
                export_data['simulation_results'][track_name] = {
                    'track_name': result['track_name'],
                    'track_file': result['track_file'],
                    'total_time_s': result['total_time_s'],
                    'total_distance_km': result['total_distance_km'],
                    'average_speed_kmh': result['average_speed_kmh'],
                    'energy_consumed_kWh': result['energy_consumed_kWh'],
                }
        
        # Calculate and add summary statistics
        successful_results = [r for r in simulation_results.values() if 'error' not in r]
        if successful_results:
            export_data['summary_statistics'] = {
                'total_tracks_attempted': len(simulation_results),
                'successful_simulations': len(successful_results),
                'failed_simulations': len(simulation_results) - len(successful_results),
                'average_speed_kmh': float(np.mean([r['average_speed_kmh'] for r in successful_results])),
                'total_distance_km': float(sum([r['total_distance_km'] for r in successful_results])),
                'total_energy_consumed_kWh': float(sum([r['energy_consumed_kWh'] for r in successful_results])),
                'average_efficiency_kWh_per_km': float(sum([r['energy_consumed_kWh'] for r in successful_results]) / 
                                                      sum([r['total_distance_km'] for r in successful_results])) if successful_results else 0
            }
        
        # Generate filename
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        motorcycle_name = current_motorcycle.name.replace(' ', '_').lower()
        filename = f"ithaka_simulation_{motorcycle_name}_{timestamp}.json"
        
        # Write to file
        with open(filename, 'w') as f:
            json.dump(export_data, f, indent=2, default=str)
        
        with export_output:
            clear_output()
            print(f"✅ Results exported successfully!")
            print(f"📄 File: {filename}")
            print(f"📊 Contains:")
            print(f"  • Complete motorcycle specifications")
            print(f"  • {len(simulation_results)} track simulation results")
            print(f"  • Summary statistics and performance metrics")
            print(f"  • Export timestamp and metadata")
            
    except Exception as e:
        with export_output:
            clear_output()
            print(f"❌ Export failed: {str(e)}")

export_json_btn.on_click(export_results_to_json)

if simulation_results:
    display(export_json_btn)
    display(export_output)
else:
    print("⚠️ Run simulation first to enable export")